In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold
from datetime import datetime

# Reading the actual Train and Test csv

In [ ]:
train_df = pd.read_csv("../data/train.csv")
train_df = train_df.drop(['GAS_H', 'RMNG_OIL_H'], axis=1)
test_df = pd.read_csv("../data/test_data.csv")

# Create a K-Fold

In [ ]:
# 5 Fold CV for the training data

train_df['fold'] = -1

np.random.seed(100)
n_splits = 5
gfold = GroupKFold(n_splits=n_splits)
gfold_loop = gfold.split(train_df, groups=train_df.CMPL_FAC_ID)

for i, (train_index, valid_index) in enumerate(gfold_loop):
    train_df.loc[valid_index, ['fold']] = i
    
train_df.to_csv("../data/train_data_with_fold.csv", index=False)

print(train_df.fold.value_counts())

# Reading the Train csv with fold and the Test csv

In [ ]:
train_df = pd.read_csv("../data/train_data_with_fold.csv")
test_df = pd.read_csv("../data/test_data.csv")

train_df['SURV_DTE'] = pd.to_datetime(train_df['SURV_DTE'])
test_df['SURV_DTE'] = pd.to_datetime(test_df['SURV_DTE'])
train_df

In [ ]:
train_df[['CMPL_FAC_ID', 
          'SAND', 
#           'SURV_DTE', 
          'AVG_ORIG_OIL_SAT', 
          'ORIG_OIL_H']].drop_duplicates().sort_values(['CMPL_FAC_ID', 'SAND']).tail(60)

# Feature Engineering

#### This portion of the code details with feature engineering

In [ ]:
def get_steam_injection_by_dist(df):
    
    df['steam_injection_by_dist_1'] = (df['FT_DIST_PAT_1']/df['SGMT_CUM_STM_INJ_1']).replace({np.inf:0, -np.inf:0})
    df['steam_injection_by_dist_2'] = (df['FT_DIST_PAT_2']/df['SGMT_CUM_STM_INJ_2']).replace({np.inf:0, -np.inf:0})
    df['steam_injection_by_dist_3'] = (df['FT_DIST_PAT_3']/df['SGMT_CUM_STM_INJ_3']).replace({np.inf:0, -np.inf:0})

    return df

train_df = get_steam_injection_by_dist(df=train_df)
test_df = get_steam_injection_by_dist(df=test_df)

train_df

In [ ]:
def get_overall_injection_per_day(df):
    df['total_injection'] = df[['SGMT_CUM_STM_INJ_1', 
                                'SGMT_CUM_STM_INJ_2', 
                                'SGMT_CUM_STM_INJ_3']].sum(axis=1)
    df['overall_injection'] = df.groupby(['CMPL_FAC_ID', 
                                          'SURV_DTE']).total_injection.transform('sum')
    df['total_injection_INJ_perc_1'] = (df['total_injection']/df['overall_injection']).replace({np.inf:0, -np.inf:0})
    df['total_injection_INJ_perc_2'] = (df['total_injection']/df['overall_injection']).replace({np.inf:0, -np.inf:0})
    df['total_injection_INJ_perc_3'] = (df['total_injection']/df['overall_injection']).replace({np.inf:0, -np.inf:0})
    
    df = df.drop(['total_injection'], axis=1)
    
    return df

train_df = get_overall_injection_per_day(df=train_df)
test_df = get_overall_injection_per_day(df=test_df)

train_df

In [ ]:
def get_min_max_dip_difference(train, test):
    
    try:
        train.drop(['fe_min_dip_diff', 'fe_max_dip_diff'], axis=1, inplace=True)
        test.drop(['fe_min_dip_diff', 'fe_max_dip_diff'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
    
    global DIP_min_max_df
    DIP_min_max_df = train[['SAND', 'DIP']].append(test[['SAND', 'DIP']])

    DIP_min_max_df = DIP_min_max_df.groupby(['SAND']).agg(min_dip = ('DIP', 'min'), 
                                                          max_dip = ('DIP', 'max')
                                                         ).reset_index()
    
    train = train.merge(DIP_min_max_df, 
                        left_on='SAND', 
                        right_on='SAND', 
                        how='left')
    train['fe_min_dip_diff'] = train['DIP'] - train['min_dip']
    train['fe_max_dip_diff'] = train['max_dip'] - train['DIP']
    
    test = test.merge(DIP_min_max_df, 
                        left_on='SAND', 
                        right_on='SAND', 
                        how='left')
    test['fe_min_dip_diff'] = test['DIP'] - test['min_dip']
    test['fe_max_dip_diff'] = test['max_dip'] - test['DIP']
        
    train.drop(['min_dip', 'max_dip'], axis=1, inplace=True)
    test.drop(['min_dip', 'max_dip'], axis=1, inplace=True)
    
    return train, test

train_df, test_df = get_min_max_dip_difference(train=train_df, test=test_df)
train_df

In [ ]:
def get_avg_target(train, test):
    
    try:
        train.drop(['fe_avg_PCT_DESAT_TO_ORIG_lag'], axis=1, inplace=True)
        test.drop(['fe_avg_PCT_DESAT_TO_ORIG_lag'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
        
    train_temp = train[['SAND', 'SURV_DTE', 'PCT_DESAT_TO_ORIG']].copy()
    train_temp = train_temp.sort_values(['SAND', 'SURV_DTE']).reset_index(drop=True)
    train_temp_1 = train_temp.groupby(['SAND', 
                                       'SURV_DTE']).agg(avg_PCT_DESAT_TO_ORIG = ('PCT_DESAT_TO_ORIG', 'mean')).reset_index()
    print(f"Train shape: {train_temp_1.shape}")
    
    test_temp = test[['SAND', 'SURV_DTE']].copy()
    test_temp = test_temp.sort_values(['SAND', 'SURV_DTE']).drop_duplicates().reset_index(drop=True)
    print(f"Test shape: {test_temp.shape}")
    
    train_test_temp_1 = train_temp_1.append(test_temp).drop_duplicates().sort_values(['SAND', 'SURV_DTE']).reset_index(drop=True)
    print(f"Train Test shape: {train_test_temp_1.shape}")

    
    global train_test_temp_2
    test_dup_condition = ((train_test_temp_1[['SAND', 'SURV_DTE']].duplicated())
                          & (train_test_temp_1.avg_PCT_DESAT_TO_ORIG.isnull()))
    train_test_temp_2 = train_test_temp_1[~test_dup_condition].reset_index(drop=True)
    print(f"Final df shape {train_test_temp_2.shape}")
    
    train_test_temp_2['avg_PCT_DESAT_TO_ORIG'] = train_test_temp_2.groupby(['SAND']).avg_PCT_DESAT_TO_ORIG.fillna(method='ffill')
    train_test_temp_2['fe_avg_PCT_DESAT_TO_ORIG_lag'] = train_test_temp_2.groupby(['SAND']).avg_PCT_DESAT_TO_ORIG.shift(1)

    train_test_temp_2 = train_test_temp_2.drop(['avg_PCT_DESAT_TO_ORIG'], axis=1)
    
    train = train.merge(train_test_temp_2, 
                        left_on = ['SAND', 'SURV_DTE'],
                        right_on = ['SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(train_test_temp_2, 
                      left_on = ['SAND', 'SURV_DTE'], 
                      right_on = ['SAND', 'SURV_DTE'], 
                      how='left')    
    print(train.shape, test.shape)
    
    return train, test

train_df, test_df = get_avg_target(train=train_df, test=test_df)

In [ ]:
def get_avg_target(train, test):
    
    try:
        train.drop(['fe_avg_sand_PCT_DESAT_TO_ORIG'], axis=1, inplace=True)
        test.drop(['fe_avg_sand_PCT_DESAT_TO_ORIG'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")

    avg_target_df = train.groupby(['SAND']).agg(fe_avg_sand_PCT_DESAT_TO_ORIG = ('PCT_DESAT_TO_ORIG', 'mean')).reset_index()

    train_df = train.merge(avg_target_df, left_on = ['SAND'], right_on = ['SAND'], how='left')
    test_df = test.merge(avg_target_df, left_on = ['SAND'], right_on = ['SAND'], how='left')
    
    return train_df, test_df

# train_df, test_df = get_avg_target(train=train_df, test=test_df)

In [ ]:
def inj_prod_distance_prod(df):
#     df['fe_Lin_dist_inj_prod_factor'] = df.Lin_Dist_Inj_Factor * df.Lin_Dist_Prod_Factor
    df['fe_Lin_dist_inj_prod_percent'] = df.Lin_Dist_Inj_Factor / df.Lin_Dist_Prod_Factor
    df['fe_Lin_dist_inj_prod_diff'] = df.Lin_Dist_Inj_Factor - df.Lin_Dist_Prod_Factor
    
    return df

train_df = inj_prod_distance_prod(df=train_df)
test_df = inj_prod_distance_prod(df=test_df)

In [ ]:
def get_oil_sat_oil_height(df):
    df['fe_oil_volume'] = df['AVG_ORIG_OIL_SAT']*df['ORIG_OIL_H']
    
    return df

train_df = get_oil_sat_oil_height(df=train_df)
test_df = get_oil_sat_oil_height(df=test_df)

In [ ]:
def get_oil_sat_oil_dip(df):
    df['fe_oil_sat_DIP'] = df['AVG_ORIG_OIL_SAT']*df['DIP']
    
    return df

train_df = get_oil_sat_oil_dip(df=train_df)
test_df = get_oil_sat_oil_dip(df=test_df)

In [ ]:
def get_total_prod_dip(df):
    df['fe_total_prod_dip'] = df['TOTAL_PROD']*df['DIP']
    
    return df

train_df = get_total_prod_dip(df=train_df)
test_df = get_total_prod_dip(df=test_df)

In [ ]:
def get_Lin_Dist_prod_Factor_dip(df):
    df['fe_Lin_Dist_prod_Factor_dip'] = df['Lin_Dist_Prod_Factor']*df['DIP']
    
    return df

train_df = get_Lin_Dist_prod_Factor_dip(df=train_df)
test_df = get_Lin_Dist_prod_Factor_dip(df=test_df)

In [ ]:
def get_SAND_cummean_DIP(train, test):
    
    try:
        train.drop(['fe_SAND_DIP_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_SAND_DIP_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'DIP']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'DIP']].copy()
    
    global avg_cummean_DIP_df
    avg_cummean_DIP_df = train_temp.append(test_temp).sort_values(['SAND',
                                                                   'SURV_DTE',
                                                                   'CMPL_FAC_ID']).reset_index(drop=True)
    avg_cummean_DIP_df['fe_SAND_DIP_cum_mean'] = avg_cummean_DIP_df.groupby(['SAND']).DIP.expanding().mean().values
#     avg_cummean_DIP_df['fe_SAND_DIP_cum_min'] = avg_cummean_DIP_df.groupby(['SAND']).DIP.expanding().min().values
#     avg_cummean_DIP_df['fe_SAND_DIP_cum_max'] = avg_cummean_DIP_df.groupby(['SAND']).DIP.expanding().max().values

    avg_cummean_DIP_df = avg_cummean_DIP_df.drop(['DIP'], axis=1)
    
    train = train.merge(avg_cummean_DIP_df, 
                        left_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                        right_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_DIP_df, 
                      left_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                      right_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_SAND_cummean_DIP(train=train_df, test=test_df)

In [ ]:
def get_Well_cummean_DIP(train, test):
    
    try:
        train.drop(['fe_Well_DIP_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_Well_DIP_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'DIP']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'DIP']].copy()
    
    global avg_cummean_DIP_df
    avg_cummean_DIP_df = train_temp.append(test_temp).sort_values(['CMPL_FAC_ID',
                                                                   'SURV_DTE',
                                                                   'SAND']).reset_index(drop=True)
    avg_cummean_DIP_df['fe_Well_DIP_cum_mean'] = avg_cummean_DIP_df.groupby(['CMPL_FAC_ID']).DIP.expanding().mean().values
#     avg_cummean_DIP_df['fe_Well_DIP_cum_min'] = avg_cummean_DIP_df.groupby(['CMPL_FAC_ID']).DIP.expanding().min().values
#     avg_cummean_DIP_df['fe_Well_DIP_cum_max'] = avg_cummean_DIP_df.groupby(['CMPL_FAC_ID']).DIP.expanding().max().values

    avg_cummean_DIP_df = avg_cummean_DIP_df.drop(['DIP'], axis=1)
    
    train = train.merge(avg_cummean_DIP_df, 
                        left_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                        right_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_DIP_df, 
                      left_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                      right_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_Well_cummean_DIP(train=train_df, test=test_df)

In [ ]:
def well_sand_cummean_AVG_ORIG_OIL_SAT(train, test):
    try:
        train.drop(['fe_Well_Sand_cummean_AVG_ORIG_OIL_SAT'], axis=1, inplace=True)
        test.drop(['fe_Well_Sand_cummean_AVG_ORIG_OIL_SAT'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['CMPL_FAC_ID', 'SAND', 'SURV_DTE', 'AVG_ORIG_OIL_SAT']].copy()
    test_temp =  test[['CMPL_FAC_ID', 'SAND', 'SURV_DTE', 'AVG_ORIG_OIL_SAT']].copy()
    
    global well_sand_AVG_ORIG_OIL_SAT_df
    
    well_sand_AVG_ORIG_OIL_SAT_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    well_sand_AVG_ORIG_OIL_SAT_df = well_sand_AVG_ORIG_OIL_SAT_df.sort_values(['CMPL_FAC_ID', 'SAND', 'SURV_DTE']).reset_index(drop=True)
    
    well_sand_AVG_ORIG_OIL_SAT_df['fe_Well_Sand_cummean_AVG_ORIG_OIL_SAT'] = well_sand_AVG_ORIG_OIL_SAT_df.groupby(['CMPL_FAC_ID', 'SAND']).AVG_ORIG_OIL_SAT.expanding().mean().values
    
    well_sand_AVG_ORIG_OIL_SAT_df = well_sand_AVG_ORIG_OIL_SAT_df.drop(['AVG_ORIG_OIL_SAT'], axis=1)
    
    train = train.merge(well_sand_AVG_ORIG_OIL_SAT_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                        how='left')
    test = test.merge(well_sand_AVG_ORIG_OIL_SAT_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                      how='left')
    
    return train, test
    
train_df, test_df = well_sand_cummean_AVG_ORIG_OIL_SAT(train=train_df, test=test_df)

In [ ]:
def get_cummean_sand_AVG_ORIG_OIL_SAT(train, test):
    
    try:
        train.drop(['fe_AVG_ORIG_OIL_SAT_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_AVG_ORIG_OIL_SAT_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'AVG_ORIG_OIL_SAT']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'AVG_ORIG_OIL_SAT']].copy()
    
    avg_orig_oil_sat_df = train_temp.append(test_temp).sort_values(['SAND', 
                                                                    'SURV_DTE',
                                                                    'CMPL_FAC_ID']).reset_index(drop=True)
    avg_orig_oil_sat_df['fe_AVG_ORIG_OIL_SAT_cum_mean'] = avg_orig_oil_sat_df.groupby(['SAND']).AVG_ORIG_OIL_SAT.expanding().mean().values

    avg_orig_oil_sat_df = avg_orig_oil_sat_df.drop(['AVG_ORIG_OIL_SAT'], axis=1)
    
    train = train.merge(avg_orig_oil_sat_df, 
                        left_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                        right_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_orig_oil_sat_df, 
                      left_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                      right_on=['SAND', 'CMPL_FAC_ID', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_sand_AVG_ORIG_OIL_SAT(train=train_df, test=test_df)

In [ ]:
# train_df['fe_pipe_avg_AVG_ORIG_OIL_SAT'] = train_df.groupby(['CMPL_FAC_ID']).AVG_ORIG_OIL_SAT.transform('mean')
# test_df['fe_pipe_avg_AVG_ORIG_OIL_SAT'] = test_df.groupby(['CMPL_FAC_ID']).AVG_ORIG_OIL_SAT.transform('mean')

def get_cummean_pipe_AVG_ORIG_OIL_SAT(train, test):
    
    try:
        train.drop(['fe_pipe_avg_AVG_ORIG_OIL_SAT_cum_mean',], axis=1, inplace=True)
        test.drop(['fe_pipe_avg_AVG_ORIG_OIL_SAT_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'AVG_ORIG_OIL_SAT']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'AVG_ORIG_OIL_SAT']].copy()
    
    global avg_cummean_AVG_ORIG_OIL_SAT_df
    avg_cummean_AVG_ORIG_OIL_SAT_df = train_temp.append(test_temp).sort_values(['CMPL_FAC_ID',
                                                                                'SURV_DTE',
                                                                                'SAND'
                                                                                ]).reset_index(drop=True)
    avg_cummean_AVG_ORIG_OIL_SAT_df['fe_pipe_avg_AVG_ORIG_OIL_SAT_cum_mean'] = avg_cummean_AVG_ORIG_OIL_SAT_df.groupby(['CMPL_FAC_ID']).AVG_ORIG_OIL_SAT.expanding().mean().values
    
    avg_cummean_AVG_ORIG_OIL_SAT_df = avg_cummean_AVG_ORIG_OIL_SAT_df.drop(['AVG_ORIG_OIL_SAT'], axis=1)
    
    train = train.merge(avg_cummean_AVG_ORIG_OIL_SAT_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_AVG_ORIG_OIL_SAT_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_pipe_AVG_ORIG_OIL_SAT(train=train_df, test=test_df)

In [ ]:
#######################################################
def get_cummean_pipe_ORIG_OIL_H(train, test):
    
    try:
        train.drop(['fe_pipe_avg_ORIG_OIL_H_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_pipe_avg_ORIG_OIL_H_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'ORIG_OIL_H']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'ORIG_OIL_H']].copy()
    
    global avg_cummean_ORIG_OIL_H_df
    avg_cummean_ORIG_OIL_H_df = train_temp.append(test_temp).sort_values(['CMPL_FAC_ID',
                                                                          'SURV_DTE',
                                                                          'SAND'
                                                                         ]).reset_index(drop=True)
    avg_cummean_ORIG_OIL_H_df['fe_pipe_avg_ORIG_OIL_H_cum_mean'] = avg_cummean_ORIG_OIL_H_df.groupby(['CMPL_FAC_ID']).ORIG_OIL_H.expanding().mean().values

    avg_cummean_ORIG_OIL_H_df = avg_cummean_ORIG_OIL_H_df.drop(['ORIG_OIL_H'], axis=1)
    
    train = train.merge(avg_cummean_ORIG_OIL_H_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_ORIG_OIL_H_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_pipe_ORIG_OIL_H(train=train_df, test=test_df)

In [ ]:
#######################################################
def get_cummean_sand_ORIG_OIL_H(train, test):
    
    try:
        train.drop(['fe_sand_ORIG_OIL_H_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_sand_ORIG_OIL_H_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'ORIG_OIL_H']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'ORIG_OIL_H']].copy()
    
    global avg_cummean_ORIG_OIL_H_df
    avg_cummean_ORIG_OIL_H_df = train_temp.append(test_temp).sort_values(['SAND',
                                                                          'SURV_DTE',
                                                                          'CMPL_FAC_ID'
                                                                         ]).reset_index(drop=True)
    avg_cummean_ORIG_OIL_H_df['fe_sand_ORIG_OIL_H_cum_mean'] = avg_cummean_ORIG_OIL_H_df.groupby(['SAND']).ORIG_OIL_H.expanding().mean().values

    avg_cummean_ORIG_OIL_H_df = avg_cummean_ORIG_OIL_H_df.drop(['ORIG_OIL_H'], axis=1)
    
    train = train.merge(avg_cummean_ORIG_OIL_H_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_ORIG_OIL_H_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_sand_ORIG_OIL_H(train=train_df, test=test_df)

In [ ]:
def get_cummean_pipe_TOTAL_INJ(train, test):
    
    try:
        train.drop(['fe_pipe_avg_TOTAL_INJ_cum_mean',
                    'fe_pipe_avg_TOTAL_INJ_cum_sum'
                    'fe_pipe_avg_TOTAL_INJ_cum_min', 
                    'fe_pipe_avg_TOTAL_INJ_cum_max'], axis=1, inplace=True)
        test.drop(['fe_pipe_avg_TOTAL_INJ_cum_mean',
                   'fe_pipe_avg_TOTAL_INJ_cum_sum',
                   'fe_pipe_avg_TOTAL_INJ_cum_min', 
                   'fe_pipe_avg_TOTAL_INJ_cum_max'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'TOTAL_INJ']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'TOTAL_INJ']].copy()
    
    global avg_cummean_TOTAL_INJ_df
    avg_cummean_TOTAL_INJ_df = train_temp.append(test_temp).sort_values(['CMPL_FAC_ID',
                                                                                'SURV_DTE',
                                                                                'SAND'
                                                                                ]).reset_index(drop=True)
    avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_cum_mean'] = avg_cummean_TOTAL_INJ_df.groupby(['CMPL_FAC_ID']).TOTAL_INJ.expanding().mean().values
    avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_cum_sum'] = avg_cummean_TOTAL_INJ_df.groupby(['CMPL_FAC_ID']).TOTAL_INJ.expanding().sum().values
#     avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_cum_min'] = avg_cummean_TOTAL_INJ_df.groupby(['CMPL_FAC_ID']).TOTAL_INJ.expanding().min().values
#     avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_cum_max'] = avg_cummean_TOTAL_INJ_df.groupby(['CMPL_FAC_ID']).TOTAL_INJ.expanding().max().values
    
        # Rolling Mean
    avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_rol3_mean'] = avg_cummean_TOTAL_INJ_df.groupby(['CMPL_FAC_ID']).TOTAL_INJ.rolling(3).mean().values
    avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_rol3_mean'] = np.where(avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_rol3_mean'].isnull(), 
                                                                           avg_cummean_TOTAL_INJ_df.TOTAL_INJ,
                                                                           avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_rol3_mean'])
    
    avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_rol5_mean'] = avg_cummean_TOTAL_INJ_df.groupby(['CMPL_FAC_ID']).TOTAL_INJ.rolling(5).mean().values
    avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_rol5_mean'] = np.where(avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_rol5_mean'].isnull(), 
                                                                           avg_cummean_TOTAL_INJ_df.TOTAL_INJ,
                                                                           avg_cummean_TOTAL_INJ_df['fe_pipe_avg_TOTAL_INJ_rol5_mean'])

    
    avg_cummean_TOTAL_INJ_df = avg_cummean_TOTAL_INJ_df.drop(['TOTAL_INJ'], axis=1)
    
    train = train.merge(avg_cummean_TOTAL_INJ_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_TOTAL_INJ_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_pipe_TOTAL_INJ(train=train_df, test=test_df)

In [ ]:
def get_cummean_sand_TOTAL_INJ(train, test):
    
    try:
        train.drop(['fe_sand_avg_TOTAL_INJ_cum_mean', 
                    'fe_sand_avg_TOTAL_INJ_cum_sum', 
                    'fe_sand_avg_TOTAL_INJ_cum_std',
                    'fe_sand_avg_TOTAL_INJ_cum_min', 
                    'fe_sand_avg_TOTAL_INJ_cum_max'], axis=1, inplace=True)
        test.drop(['fe_sand_avg_TOTAL_INJ_cum_mean',
                   'fe_sand_avg_TOTAL_INJ_cum_sum', 
                   'fe_sand_avg_TOTAL_INJ_cum_std',
                   'fe_sand_avg_TOTAL_INJ_cum_min', 
                   'fe_sand_avg_TOTAL_INJ_cum_max'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'TOTAL_INJ']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'TOTAL_INJ']].copy()
    
    global avg_cummean_TOTAL_INJ_df
    avg_cummean_TOTAL_INJ_df = train_temp.append(test_temp).sort_values(['SAND',
                                                                         'SURV_DTE',
                                                                         'CMPL_FAC_ID'
                                                                        ]).reset_index(drop=True)
    avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_cum_mean'] = avg_cummean_TOTAL_INJ_df.groupby(['SAND']).TOTAL_INJ.expanding().mean().values   
    avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_cum_sum'] = avg_cummean_TOTAL_INJ_df.groupby(['SAND']).TOTAL_INJ.expanding().sum().values
    avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_cum_std'] = avg_cummean_TOTAL_INJ_df.groupby(['SAND']).TOTAL_INJ.expanding().std().values
    
    # Rolling Mean
    avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_rol3_mean'] = avg_cummean_TOTAL_INJ_df.groupby(['SAND']).TOTAL_INJ.rolling(3).mean().values
    avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_rol3_mean'] = np.where(avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_rol3_mean'].isnull(), 
                                                                           avg_cummean_TOTAL_INJ_df.TOTAL_INJ,
                                                                           avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_rol3_mean'])
    
    avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_rol5_mean'] = avg_cummean_TOTAL_INJ_df.groupby(['SAND']).TOTAL_INJ.rolling(5).mean().values
    avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_rol5_mean'] = np.where(avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_rol5_mean'].isnull(), 
                                                                           avg_cummean_TOTAL_INJ_df.TOTAL_INJ,
                                                                           avg_cummean_TOTAL_INJ_df['fe_sand_avg_TOTAL_INJ_rol5_mean'])
    
    avg_cummean_TOTAL_INJ_df = avg_cummean_TOTAL_INJ_df.drop(['TOTAL_INJ'], axis=1)
    
    train = train.merge(avg_cummean_TOTAL_INJ_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_TOTAL_INJ_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_sand_TOTAL_INJ(train=train_df, test=test_df)

In [ ]:
def get_cummean_pipe_TOTAL_GNTL_INJ(train, test):
    
    try:
        train.drop(['fe_pipe_avg_TOTAL_GNTL_INJ_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_pipe_avg_TOTAL_GNTL_INJ_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'TOTAL_GNTL_INJ']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'TOTAL_GNTL_INJ']].copy()
    
    global avg_cummean_TOTAL_GNTL_INJ_df
    avg_cummean_TOTAL_GNTL_INJ_df = train_temp.append(test_temp).sort_values(['CMPL_FAC_ID',
                                                                                'SURV_DTE',
                                                                                'SAND'
                                                                                ]).reset_index(drop=True)
    avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_cum_mean'] = avg_cummean_TOTAL_GNTL_INJ_df.groupby(['CMPL_FAC_ID']).TOTAL_GNTL_INJ.expanding().mean().values
    
    # Rolling Mean
    avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_rol3_mean'] = avg_cummean_TOTAL_GNTL_INJ_df.groupby(['SAND']).TOTAL_GNTL_INJ.rolling(3).mean().values
    avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_rol3_mean'] = np.where(avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_rol3_mean'].isnull(), 
                                                                                     avg_cummean_TOTAL_GNTL_INJ_df.TOTAL_GNTL_INJ,
                                                                                     avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_rol3_mean'])
    
    avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_rol5_mean'] = avg_cummean_TOTAL_GNTL_INJ_df.groupby(['SAND']).TOTAL_GNTL_INJ.rolling(5).mean().values
    avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_rol5_mean'] = np.where(avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_rol5_mean'].isnull(), 
                                                                                     avg_cummean_TOTAL_GNTL_INJ_df.TOTAL_GNTL_INJ,
                                                                                     avg_cummean_TOTAL_GNTL_INJ_df['fe_pipe_avg_TOTAL_GNTL_INJ_rol5_mean'])

    
    avg_cummean_TOTAL_GNTL_INJ_df = avg_cummean_TOTAL_GNTL_INJ_df.drop(['TOTAL_GNTL_INJ'], axis=1)
    
    train = train.merge(avg_cummean_TOTAL_GNTL_INJ_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_TOTAL_GNTL_INJ_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_pipe_TOTAL_GNTL_INJ(train=train_df, test=test_df)

In [ ]:
def get_cummean_sand_TOTAL_GNTL_INJ(train, test):
    
    try:
        train.drop(['fe_sand_avg_TOTAL_GNTL_INJ_cum_mean',
                    'fe_sand_avg_TOTAL_GNTL_INJ_cum_sum'], axis=1, inplace=True)
        test.drop(['fe_sand_avg_TOTAL_GNTL_INJ_cum_mean',
                   'fe_sand_avg_TOTAL_GNTL_INJ_cumsum'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'TOTAL_GNTL_INJ']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'TOTAL_GNTL_INJ']].copy()
    
    global avg_cummean_TOTAL_GNTL_INJ_df
    avg_cummean_TOTAL_GNTL_INJ_df = train_temp.append(test_temp).sort_values(['SAND',
                                                                                'SURV_DTE',
                                                                                'CMPL_FAC_ID'
                                                                                ]).reset_index(drop=True)
    avg_cummean_TOTAL_GNTL_INJ_df['fe_sand_avg_TOTAL_GNTL_INJ_cum_mean'] = avg_cummean_TOTAL_GNTL_INJ_df.groupby(['SAND']).TOTAL_GNTL_INJ.expanding().mean().values
    avg_cummean_TOTAL_GNTL_INJ_df['fe_sand_avg_TOTAL_GNTL_INJ_cum_sum'] = avg_cummean_TOTAL_GNTL_INJ_df.groupby(['SAND']).TOTAL_GNTL_INJ.expanding().sum().values
    
    avg_cummean_TOTAL_GNTL_INJ_df = avg_cummean_TOTAL_GNTL_INJ_df.drop(['TOTAL_GNTL_INJ'], axis=1)
    
    train = train.merge(avg_cummean_TOTAL_GNTL_INJ_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_TOTAL_GNTL_INJ_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_sand_TOTAL_GNTL_INJ(train=train_df, test=test_df)

In [ ]:
def get_cummean_PCT_DESAT_TO_ORIG(train, test):
    
    try:
        train.drop(['fe_PCT_DESAT_TO_ORIG_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_PCT_DESAT_TO_ORIG_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'PCT_DESAT_TO_ORIG']].copy()
    
    global avg_PCT_DESAT_TO_ORIG_df
    avg_PCT_DESAT_TO_ORIG_df = train_temp.sort_values(['SAND',                                                                     
                                                       'SURV_DTE',
                                                       'CMPL_FAC_ID']).reset_index(drop=True)
    avg_PCT_DESAT_TO_ORIG_df['fe_PCT_DESAT_TO_ORIG_cum_mean'] = avg_PCT_DESAT_TO_ORIG_df.groupby(['SAND']).PCT_DESAT_TO_ORIG.expanding().mean().values

    avg_PCT_DESAT_TO_ORIG_df = avg_PCT_DESAT_TO_ORIG_df.drop(['PCT_DESAT_TO_ORIG'], axis=1)
    
    train = train.merge(avg_PCT_DESAT_TO_ORIG_df, 
                        left_on=['SAND'], 
                        right_on=['SAND'], 
                        how='left')
    test = test.merge(avg_PCT_DESAT_TO_ORIG_df, 
                      left_on=['SAND'], 
                      right_on=['SAND'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_AVG_ORIG_OIL_SAT(train=train_df, test=test_df)

In [ ]:
def get_cummean_pipe_TOTAL_PROD(train, test):
    
    try:
        train.drop(['fe_pipe_TOTAL_PROD_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_pipe_TOTAL_PROD_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'TOTAL_PROD']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'TOTAL_PROD']].copy()
    
    global avg_cummean_TOTAL_PROD_df
    avg_cummean_TOTAL_PROD_df = train_temp.append(test_temp).sort_values(['CMPL_FAC_ID',
                                                                          'SURV_DTE',
                                                                          'SAND'
                                                                         ]).reset_index(drop=True)
    avg_cummean_TOTAL_PROD_df['fe_pipe_TOTAL_PROD_cum_mean'] = avg_cummean_TOTAL_PROD_df.groupby(['CMPL_FAC_ID']).TOTAL_PROD.expanding().mean().values

    
    # Rolling Mean
    avg_cummean_TOTAL_PROD_df['fe_pipe_avg_TOTAL_PROD_rol3_mean'] = avg_cummean_TOTAL_PROD_df.groupby(['CMPL_FAC_ID']).TOTAL_PROD.rolling(3).mean().values
    avg_cummean_TOTAL_PROD_df['fe_pipe_avg_TOTAL_PROD_rol3_mean'] = np.where(avg_cummean_TOTAL_PROD_df['fe_pipe_avg_TOTAL_PROD_rol3_mean'].isnull(), 
                                                                             avg_cummean_TOTAL_PROD_df.TOTAL_PROD,
                                                                             avg_cummean_TOTAL_PROD_df['fe_pipe_avg_TOTAL_PROD_rol3_mean'])

    avg_cummean_TOTAL_PROD_df['fe_pipe_avg_TOTAL_PROD_rol5_mean'] = avg_cummean_TOTAL_PROD_df.groupby(['CMPL_FAC_ID']).TOTAL_PROD.rolling(5).mean().values
    avg_cummean_TOTAL_PROD_df['fe_pipe_avg_TOTAL_PROD_rol5_mean'] = np.where(avg_cummean_TOTAL_PROD_df['fe_pipe_avg_TOTAL_PROD_rol3_mean'].isnull(), 
                                                                             avg_cummean_TOTAL_PROD_df.TOTAL_PROD,
                                                                             avg_cummean_TOTAL_PROD_df['fe_pipe_avg_TOTAL_PROD_rol3_mean'])
    
    avg_cummean_TOTAL_PROD_df = avg_cummean_TOTAL_PROD_df.drop(['TOTAL_PROD'], axis=1)
    
    train = train.merge(avg_cummean_TOTAL_PROD_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_TOTAL_PROD_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_pipe_TOTAL_PROD(train=train_df, test=test_df)

In [ ]:
def get_cummean_sand_TOTAL_PROD(train, test):
    
    try:
        train.drop(['fe_sand_TOTAL_PROD_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_sand_TOTAL_PROD_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'TOTAL_PROD']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'TOTAL_PROD']].copy()
    
    global avg_cummean_TOTAL_PROD_df
    avg_cummean_TOTAL_PROD_df = train_temp.append(test_temp).sort_values(['SAND',
                                                                          'SURV_DTE',
                                                                          'CMPL_FAC_ID'
                                                                         ]).reset_index(drop=True)
    avg_cummean_TOTAL_PROD_df['fe_sand_TOTAL_PROD_cum_mean'] = avg_cummean_TOTAL_PROD_df.groupby(['SAND']).TOTAL_PROD.expanding().mean().values

    # Rolling Mean
    avg_cummean_TOTAL_PROD_df['fe_sand_avg_TOTAL_PROD_rol3_mean'] = avg_cummean_TOTAL_PROD_df.groupby(['SAND']).TOTAL_PROD.rolling(3).mean().values
    avg_cummean_TOTAL_PROD_df['fe_sand_avg_TOTAL_PROD_rol3_mean'] = np.where(avg_cummean_TOTAL_PROD_df['fe_sand_avg_TOTAL_PROD_rol3_mean'].isnull(), 
                                                                             avg_cummean_TOTAL_PROD_df.TOTAL_PROD,
                                                                             avg_cummean_TOTAL_PROD_df['fe_sand_avg_TOTAL_PROD_rol3_mean'])
    
    avg_cummean_TOTAL_PROD_df['fe_sand_avg_TOTAL_PROD_rol5_mean'] = avg_cummean_TOTAL_PROD_df.groupby(['SAND']).TOTAL_PROD.rolling(5).mean().values
    avg_cummean_TOTAL_PROD_df['fe_sand_avg_TOTAL_PROD_rol5_mean'] = np.where(avg_cummean_TOTAL_PROD_df['fe_sand_avg_TOTAL_PROD_rol3_mean'].isnull(), 
                                                                             avg_cummean_TOTAL_PROD_df.TOTAL_PROD,
                                                                             avg_cummean_TOTAL_PROD_df['fe_sand_avg_TOTAL_PROD_rol3_mean'])

    
    avg_cummean_TOTAL_PROD_df = avg_cummean_TOTAL_PROD_df.drop(['TOTAL_PROD'], axis=1)
    
    train = train.merge(avg_cummean_TOTAL_PROD_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_TOTAL_PROD_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_sand_TOTAL_PROD(train=train_df, test=test_df)

In [ ]:
def well_sand_cummean_TOTAL_PROD(train, test):
    try:
        train.drop(['fe_Well_Sand_cummean_TOTAL_PROD'], axis=1, inplace=True)
        test.drop(['fe_Well_Sand_cummean_TOTAL_PROD'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['CMPL_FAC_ID', 'SAND', 'SURV_DTE', 'TOTAL_PROD']].copy()
    test_temp =  test[['CMPL_FAC_ID', 'SAND', 'SURV_DTE', 'TOTAL_PROD']].copy()
    
    global well_sand_TOTAL_PROD_df
    
    well_sand_TOTAL_PROD_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    well_sand_TOTAL_PROD_df = well_sand_TOTAL_PROD_df.sort_values(['CMPL_FAC_ID', 'SAND', 'SURV_DTE']).reset_index(drop=True)
    
    well_sand_TOTAL_PROD_df['fe_Well_Sand_cummean_TOTAL_PROD'] = well_sand_TOTAL_PROD_df.groupby(['CMPL_FAC_ID', 'SAND']).TOTAL_PROD.expanding().mean().values
    
    well_sand_TOTAL_PROD_df = well_sand_TOTAL_PROD_df.drop(['TOTAL_PROD'], axis=1)
    
    train = train.merge(well_sand_TOTAL_PROD_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                        how='left')
    test = test.merge(well_sand_TOTAL_PROD_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                      how='left')
    
    return train, test
    
train_df, test_df = well_sand_cummean_TOTAL_PROD(train=train_df, test=test_df)

In [ ]:
def total_wells(df):
    df['fe_total_wells'] = ((~df.FT_DIST_PAT_1.isnull()).astype('int') + 
                            (~df.FT_DIST_PAT_2.isnull()).astype('int') + 
                            (~df.FT_DIST_PAT_3.isnull()).astype('int')
                        )
    return df

train_df = total_wells(df=train_df)
test_df = total_wells(df=test_df)

In [ ]:
def avg_well_distance(df):
    df['fe_avg_well_distance']= df[['FT_DIST_PAT_1', 'FT_DIST_PAT_2', 'FT_DIST_PAT_3']].mean(axis=1)
    
    return df

train_df = avg_well_distance(df=train_df)
test_df = avg_well_distance(df=test_df)

In [ ]:
def total_and_avg_well_injection(df):
    
#     df['fe_sum_well_injection']= train_df[['SGMT_CUM_STM_INJ_1', 'SGMT_CUM_STM_INJ_2', 'SGMT_CUM_STM_INJ_3']].sum(axis=1)
    overall = train_df[['SGMT_CUM_STM_INJ_1', 'SGMT_CUM_STM_INJ_2', 'SGMT_CUM_STM_INJ_3']].sum(axis=1)
    df['fe_total_injected_percentage_1'] = (df['SGMT_CUM_STM_INJ_1']/overall).replace({np.inf:0})
    df['fe_total_injected_percentage_2'] = (df['SGMT_CUM_STM_INJ_2']/overall).replace({np.inf:0})
    df['fe_total_injected_percentage_3'] = (df['SGMT_CUM_STM_INJ_3']/overall).replace({np.inf:0})
    
    df['fe_avg_well_injection']= overall/3
    
#     df.drop(['fe_sum_well_injection'], axis=1, inplace=True)
    
    return df
    
train_df = total_and_avg_well_injection(df=train_df)
test_df = total_and_avg_well_injection(df=test_df)

In [ ]:
def total_wells_per_sand(train, test):
    
    try:
        train.drop(['fe_unique_well_count'], axis=1, inplace=True)
        test.drop(['fe_unique_well_count'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['SAND', 'CMPL_FAC_ID', 'SURV_DTE']].copy()
    test_temp =  test[['SAND', 'CMPL_FAC_ID', 'SURV_DTE']].copy()
    
    global uniq_well_df
    uniq_well_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    uniq_well_df = uniq_well_df.groupby(['SAND', 'CMPL_FAC_ID']).agg(min_date=('SURV_DTE', 'min')).reset_index()
    uniq_well_df = uniq_well_df.sort_values(['SAND', 'min_date', 'CMPL_FAC_ID']).reset_index(drop=True)
    uniq_well_df['fe_unique_well_count'] = uniq_well_df.groupby(['SAND']).CMPL_FAC_ID.cumcount()
        
    uniq_well_df = uniq_well_df.drop(['min_date'], axis=1)
    
    train = train.merge(uniq_well_df, 
                        left_on=['SAND', 'CMPL_FAC_ID'], 
                        right_on=['SAND', 'CMPL_FAC_ID'], 
                        how='left')
    test = test.merge(uniq_well_df, 
                      left_on=['SAND', 'CMPL_FAC_ID'], 
                      right_on=['SAND', 'CMPL_FAC_ID'], 
                      how='left')
    
    return train, test
    
train_df, test_df = total_wells_per_sand(train=train_df, test=test_df)

In [ ]:
def total_sand_per_wells(train, test):
    
    try:
        train.drop(['fe_unique_sand_per_well_count'], axis=1, inplace=True)
        test.drop(['fe_unique_sand_per_well_count'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['SAND', 'CMPL_FAC_ID', 'SURV_DTE']].copy()
    test_temp =  test[['SAND', 'CMPL_FAC_ID', 'SURV_DTE']].copy()
    
    global uniq_sand_df
    uniq_sand_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    uniq_sand_df = uniq_sand_df.groupby(['CMPL_FAC_ID', 'SAND']).agg(min_date=('SURV_DTE', 'min')).reset_index()
    uniq_sand_df = uniq_sand_df.sort_values(['CMPL_FAC_ID', 'min_date', 'SAND']).reset_index(drop=True)
    uniq_sand_df['fe_unique_sand_per_well_count'] = uniq_sand_df.groupby(['CMPL_FAC_ID']).SAND.cumcount()
        
    uniq_sand_df = uniq_sand_df.drop(['min_date'], axis=1)
    
    train = train.merge(uniq_sand_df, 
                        left_on=['CMPL_FAC_ID', 'SAND'], 
                        right_on=['CMPL_FAC_ID', 'SAND'], 
                        how='left')
    test = test.merge(uniq_sand_df, 
                      left_on=['CMPL_FAC_ID', 'SAND'], 
                      right_on=['CMPL_FAC_ID', 'SAND'], 
                      how='left')
    
    return train, test
    
train_df, test_df = total_sand_per_wells(train=train_df, test=test_df)

In [ ]:
def sand_age(train, test):
    
    train_temp =  train[['SAND', 'SURV_DTE']].copy()
    test_temp =  test[['SAND', 'SURV_DTE']].copy()
    
    global sand_min_age_df
    SAND_age_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    sand_min_age_df = SAND_age_df.groupby(['SAND']).agg(sand_min_age = ('SURV_DTE', 'min')).reset_index()
    
    train = train.merge(sand_min_age_df, 
                        left_on=['SAND'], 
                        right_on=['SAND'], 
                        how='left')
    test = test.merge(sand_min_age_df, 
                      left_on=['SAND'], 
                      right_on=['SAND'], 
                      how='left')    
    
    train['fe_SAND_AGE'] = (train.SURV_DTE - train.sand_min_age).dt.days
    test['fe_SAND_AGE'] = (test.SURV_DTE - test.sand_min_age).dt.days
    
    train.drop(['sand_min_age'], axis=1, inplace=True)
    test.drop(['sand_min_age'], axis=1, inplace=True)
    
    return train, test

train_df, test_df = sand_age(train=train_df, test=test_df)

In [ ]:
def well_age(df):
    
    well_min_age_df = df.groupby(['CMPL_FAC_ID']).agg(well_min_age = ('SURV_DTE', 'min')).reset_index()
    df = df.merge(well_min_age_df, left_on=['CMPL_FAC_ID'], right_on=['CMPL_FAC_ID'], how='left')
    df['fe_WELL_AGE'] = (df.SURV_DTE - df.well_min_age).dt.days
    df.drop(['well_min_age'], axis=1, inplace=True)
    
    return df

train_df = well_age(df=train_df)
test_df = well_age(df=test_df)

In [ ]:
def well_sand_age(df):
    
    well_sand_min_age_df = df.groupby(['SAND', 'CMPL_FAC_ID']).agg(well_sand_min_age = ('SURV_DTE', 'min')).reset_index()
    df = df.merge(well_sand_min_age_df, 
                  left_on=['SAND', 'CMPL_FAC_ID'], 
                  right_on=['SAND', 'CMPL_FAC_ID'], 
                  how='left')
    df['fe_WELL_SAND_AGE'] = (df.SURV_DTE - df.well_sand_min_age).dt.days
    df.drop(['well_sand_min_age'], axis=1, inplace=True)
    
    return df

train_df = well_sand_age(df=train_df)
test_df = well_sand_age(df=test_df)

In [ ]:
def SAND_last_active(train, test):
    try:
        train.drop(['SAND_last_active'], axis=1, inplace=True)
        test.drop(['SAND_last_active'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['SAND', 'SURV_DTE']].copy()
    test_temp =  test[['SAND', 'SURV_DTE']].copy()
    
    last_active_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    last_active_df = last_active_df.sort_values(['SAND', 'SURV_DTE']).reset_index(drop=True)
    
    last_active_df['SURV_DTE_lag'] = last_active_df.groupby(['SAND']).SURV_DTE.shift(1)
    last_active_df['fe_SAND_last_active'] = (last_active_df.SURV_DTE - last_active_df.SURV_DTE_lag).dt.days
    last_active_df = last_active_df.drop(['SURV_DTE_lag'], axis=1)
#     last_active_df = last_active_df.fillna(0)
    
    train = train.merge(last_active_df, 
                        left_on=['SAND', 'SURV_DTE'], 
                        right_on=['SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(last_active_df, 
                      left_on=['SAND', 'SURV_DTE'], 
                      right_on=['SAND', 'SURV_DTE'], 
                      how='left')
    
    return train, test
    
train_df, test_df = SAND_last_active(train=train_df, test=test_df)

In [ ]:
def WELL_last_active(train, test):
    
    try:
        train.drop(['Well_last_active'], axis=1, inplace=True)
        test.drop(['Well_last_active'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
    
    train_temp =  train[['CMPL_FAC_ID', 'SURV_DTE']].copy()
    test_temp =  test[['CMPL_FAC_ID', 'SURV_DTE']].copy()
    
    last_active_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    last_active_df = last_active_df.sort_values(['CMPL_FAC_ID', 'SURV_DTE']).reset_index(drop=True)
    
    last_active_df['SURV_DTE_lag'] = last_active_df.groupby(['CMPL_FAC_ID']).SURV_DTE.shift(1)
    last_active_df['fe_Well_last_active'] = (last_active_df.SURV_DTE - last_active_df.SURV_DTE_lag).dt.days
    last_active_df = last_active_df.drop(['SURV_DTE_lag'], axis=1)
#     last_active_df = last_active_df.fillna(0)
    
    train = train.merge(last_active_df, 
                        left_on=['CMPL_FAC_ID', 'SURV_DTE'],
                        right_on=['CMPL_FAC_ID', 'SURV_DTE'],
                        how='left')
    test = test.merge(last_active_df, 
                      left_on=['CMPL_FAC_ID', 'SURV_DTE'],
                      right_on=['CMPL_FAC_ID', 'SURV_DTE'],
                      how='left')
    
    return train, test
    
train_df, test_df = WELL_last_active(train=train_df, test=test_df)

In [ ]:
def well_sand_last_active(train, test):
    try:
        train.drop(['Well_Sand_last_active'], axis=1, inplace=True)
        test.drop(['Well_Sand_last_active'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['CMPL_FAC_ID', 'SAND', 'SURV_DTE']].copy()
    test_temp =  test[['CMPL_FAC_ID', 'SAND', 'SURV_DTE']].copy()
    
    last_active_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    last_active_df = last_active_df.sort_values(['CMPL_FAC_ID', 'SAND', 'SURV_DTE']).reset_index(drop=True)
    
    last_active_df['SURV_DTE_lag'] = last_active_df.groupby(['CMPL_FAC_ID', 'SAND']).SURV_DTE.shift(1)
    last_active_df['fe_Well_Sand_last_active'] = (last_active_df.SURV_DTE - last_active_df.SURV_DTE_lag).dt.days
    last_active_df = last_active_df.drop(['SURV_DTE_lag'], axis=1)
#     last_active_df = last_active_df.fillna(0)
    
    train = train.merge(last_active_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                        how='left')
    test = test.merge(last_active_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                      how='left')
    
    return train, test
    
train_df, test_df = well_sand_last_active(train=train_df, test=test_df)

In [ ]:
def well_sand_usage(train, test):
    try:
        train.drop(['Well_Sand_usage_count'], axis=1, inplace=True)
        test.drop(['Well_Sand_usage_count'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['CMPL_FAC_ID', 'SAND', 'SURV_DTE']].copy()
    test_temp =  test[['CMPL_FAC_ID', 'SAND', 'SURV_DTE']].copy()
    
    global well_sand_usage_df
    
    well_sand_usage_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    well_sand_usage_df = well_sand_usage_df.sort_values(['CMPL_FAC_ID', 'SAND', 'SURV_DTE']).reset_index(drop=True)
    
    well_sand_usage_df['fe_Well_Sand_usage_count'] = well_sand_usage_df.groupby(['CMPL_FAC_ID', 'SAND']).CMPL_FAC_ID.cumcount()
    
    train = train.merge(well_sand_usage_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                        how='left')
    test = test.merge(well_sand_usage_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'],
                      how='left')
    
    return train, test
    
train_df, test_df = well_sand_usage(train=train_df, test=test_df)

In [ ]:
def well_usage(train, test):
    try:
        train.drop(['Well_usage_count'], axis=1, inplace=True)
        test.drop(['Well_usage_count'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['CMPL_FAC_ID', 'SURV_DTE']].copy()
    test_temp =  test[['CMPL_FAC_ID', 'SURV_DTE']].copy()
    
    global well_usage_df
    
    well_usage_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    well_usage_df = well_usage_df.sort_values(['CMPL_FAC_ID', 'SURV_DTE']).reset_index(drop=True)
    
    well_usage_df['fe_Well_usage_count'] = well_usage_df.groupby(['CMPL_FAC_ID']).SURV_DTE.cumcount()
    
    train = train.merge(well_usage_df, 
                        left_on=['CMPL_FAC_ID', 'SURV_DTE'],
                        right_on=['CMPL_FAC_ID', 'SURV_DTE'],
                        how='left')
    test = test.merge(well_usage_df, 
                      left_on=['CMPL_FAC_ID', 'SURV_DTE'],
                      right_on=['CMPL_FAC_ID', 'SURV_DTE'],
                      how='left')
    
    return train, test
    
train_df, test_df = well_usage(train=train_df, test=test_df)

In [ ]:
def sand_usage(train, test):
    try:
        train.drop(['Sand_usage_count'], axis=1, inplace=True)
        test.drop(['Sand_usage_count'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp =  train[['SAND', 'SURV_DTE']].copy()
    test_temp =  test[['SAND', 'SURV_DTE']].copy()
    
    global sand_usage_df
    
    sand_usage_df = train_temp.append(test_temp).drop_duplicates().reset_index(drop=True)
    sand_usage_df = sand_usage_df.sort_values(['SAND', 'SURV_DTE']).reset_index(drop=True)
    
    sand_usage_df['fe_Sand_usage_count'] = sand_usage_df.groupby(['SAND']).SURV_DTE.cumcount()
    
    train = train.merge(sand_usage_df, 
                        left_on=['SAND', 'SURV_DTE'],
                        right_on=['SAND', 'SURV_DTE'],
                        how='left')
    test = test.merge(sand_usage_df, 
                      left_on=['SAND', 'SURV_DTE'],
                      right_on=['SAND', 'SURV_DTE'],
                      how='left')
    
    return train, test
    
train_df, test_df = sand_usage(train=train_df, test=test_df)

In [ ]:
def get_cummean_pipe_fe_injection_difference(train, test):
    
    try:
        train.drop(['fe_pipe_fe_injection_difference_cum_mean'], axis=1, inplace=True)
        test.drop(['fe_pipe_fe_injection_difference_cum_mean'], axis=1, inplace=True)
    except:
        print("First time")
    else:
        print("Repeat so dropped the column")
        
    train_temp = train[['SAND', 'SURV_DTE', 
                        'CMPL_FAC_ID', 'fe_injection_difference']].copy()
    test_temp = test[['SAND', 'SURV_DTE', 
                      'CMPL_FAC_ID', 'fe_injection_difference']].copy()
    
    global avg_cummean_fe_injection_difference_df
    avg_cummean_fe_injection_difference_df = train_temp.append(test_temp).sort_values(['CMPL_FAC_ID',
                                                                          'SURV_DTE',
                                                                          'SAND'
                                                                         ]).reset_index(drop=True)
    avg_cummean_fe_injection_difference_df['fe_pipe_fe_injection_difference_cum_mean'] = avg_cummean_fe_injection_difference_df.groupby(['CMPL_FAC_ID']).fe_injection_difference.expanding().mean().values

    avg_cummean_fe_injection_difference_df = avg_cummean_fe_injection_difference_df.drop(['fe_injection_difference'], axis=1)
    
    train = train.merge(avg_cummean_fe_injection_difference_df, 
                        left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                        how='left')
    test = test.merge(avg_cummean_fe_injection_difference_df, 
                      left_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      right_on=['CMPL_FAC_ID', 'SAND', 'SURV_DTE'], 
                      how='left')

    return train, test

train_df, test_df = get_cummean_pipe_fe_injection_difference(train=train_df, test=test_df)

In [ ]:
train_df['fe_sand_reservoir_percentage']  = train_df.TOTAL_GNTL_INJ/ train_df.TOTAL_INJ
test_df['fe_sand_reservoir_percentage']  = test_df.TOTAL_GNTL_INJ/ test_df.TOTAL_INJ

train_df['fe_prod_inj_percentage']  = train_df.TOTAL_PROD/ train_df.TOTAL_INJ
test_df['fe_prod_inj_percentage']  = test_df.TOTAL_PROD/ test_df.TOTAL_INJ

train_df['fe_injection_difference'] = train_df.TOTAL_INJ - train_df.TOTAL_GNTL_INJ 
test_df['fe_injection_difference'] = test_df.TOTAL_INJ - test_df.TOTAL_GNTL_INJ 

train_df['fe_injection_difference_percentage'] = (train_df.fe_injection_difference/train_df.TOTAL_GNTL_INJ).replace({np.inf:0})
test_df['fe_injection_difference_percentage'] = (test_df.fe_injection_difference/test_df.TOTAL_GNTL_INJ).replace({np.inf:0})

train_df['fe_injection_difference_reservoir_percentage'] = (train_df.fe_injection_difference/train_df.TOTAL_INJ).replace({np.inf:0, -np.inf:0})
test_df['fe_injection_difference_reservoir_percentage'] = (test_df.fe_injection_difference/test_df.TOTAL_INJ).replace({np.inf:0, -np.inf:0})

train_df['fe_injection_difference_PROD_percentage'] = (train_df.fe_injection_difference/train_df.TOTAL_PROD).replace({np.inf:0})
test_df['fe_injection_difference_PROD_percentage'] = (test_df.fe_injection_difference/test_df.TOTAL_PROD).replace({np.inf:0})

train_df['fe_injection_difference_oil_volume'] = train_df['fe_injection_difference']*train_df['ORIG_OIL_H']
test_df['fe_injection_difference_oil_volume'] = train_df['fe_injection_difference']*train_df['ORIG_OIL_H']

train_df['fe_prod_sand_inje_difference'] = train_df.TOTAL_PROD - train_df.TOTAL_GNTL_INJ
test_df['fe_prod_sand_inje_difference'] = test_df.TOTAL_PROD - test_df.TOTAL_GNTL_INJ

train_df['fe_prod_reservoir_inje_difference'] = train_df.TOTAL_PROD - train_df.TOTAL_INJ
test_df['fe_prod_reservoir_inje_difference'] = test_df.TOTAL_PROD - test_df.TOTAL_INJ

train_df['fe_prod_reservoir_inje_difference_prod_percent'] = (train_df.fe_prod_reservoir_inje_difference/train_df.TOTAL_PROD).replace({np.inf:0, -np.inf:0})
test_df['fe_prod_reservoir_inje_difference_prod_percent'] = (train_df.fe_prod_reservoir_inje_difference/train_df.TOTAL_PROD).replace({np.inf:0, -np.inf:0})

train_df['fe_prod_reservoir_inje_difference_TOTAL_INJ_percent'] = train_df.fe_prod_reservoir_inje_difference/train_df.TOTAL_INJ
test_df['fe_prod_reservoir_inje_difference_TOTAL_INJ_percent'] = train_df.fe_prod_reservoir_inje_difference/train_df.TOTAL_INJ


In [ ]:
# FIllna
train_df = train_df.fillna(-999)
test_df = test_df.fillna(-999)

# mean_cols = test_df.columns.difference(['SAND', 'SURV_DTE'])
# train_df[mean_cols] = train_df[mean_cols].apply(lambda x : x.fillna(x.mean(), axis=0))
# test_df[mean_cols]  =  test_df[mean_cols].apply(lambda x : x.fillna(x.mean(), axis=0))

# train = train_df.sort_values(['SURV_DTE', 'SAND', 'CMPL_FAC_ID']).reset_index(drop=True)
train_df

In [ ]:
# Label Encode
le = LabelEncoder()
le.fit(train_df.SAND)
train_df['SAND'] = le.transform(train_df.SAND)
test_df['SAND'] = le.transform(test_df.SAND)

In [ ]:
# Adding Neural Network embeddings as features

indep=['AVG_ORIG_OIL_SAT', 'DIP', 'FT_DIST_PAT_1', 'FT_DIST_PAT_2',
       'FT_DIST_PAT_3', 'Lin_Dist_Inj_Factor', 'Lin_Dist_Prod_Factor',
       'ORIG_OIL_H', 'SAND', 'SGMT_CUM_STM_INJ_1', 'SGMT_CUM_STM_INJ_2',
       'SGMT_CUM_STM_INJ_3', 'TOTAL_GNTL_INJ', 'TOTAL_INJ', 'TOTAL_PROD',
       'fe_AVG_ORIG_OIL_SAT_cum_mean', 'fe_SAND_AGE', 'fe_SAND_DIP_cum_mean',
       'fe_SAND_last_active', 'fe_Sand_usage_count', 'fe_WELL_AGE',
       'fe_WELL_SAND_AGE', 'fe_Well_DIP_cum_mean', 'fe_Well_Sand_last_active',
       'fe_Well_Sand_usage_count', 'fe_Well_last_active',
       'fe_Well_usage_count', 'fe_avg_well_distance', 'fe_avg_well_injection',
       'fe_injection_difference', 'fe_injection_difference_PROD_percentage',
       'fe_injection_difference_percentage', 'fe_oil_sat_DIP', 'fe_oil_volume',
       'fe_pipe_avg_AVG_ORIG_OIL_SAT_cum_mean',
       'fe_pipe_avg_ORIG_OIL_H_cum_mean', 'fe_pipe_avg_TOTAL_INJ_cum_mean',
       'fe_prod_inj_percentage', 'fe_prod_reservoir_inje_difference',
       'fe_prod_sand_inje_difference', 'fe_sand_reservoir_percentage',
       'fe_total_injected_percentage_1', 'fe_total_injected_percentage_2',
       'fe_total_injected_percentage_3', 'fe_total_wells',
       'fe_unique_well_count']

class Regression_NN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, 128)
        self.hidden_layer_1 = nn.Linear(128, 64)
        self.hidden_layer_2 = nn.Linear(64, 32)
        self.hidden_layer_3 = nn.Linear(32, 64)
        self.hidden_layer_4 = nn.Linear(64, 128)
        self.output_layer = nn.Linear(128, input_dim)
    
        self.relu = nn.ReLU()
    
    def forward(self, x):
        output = self.input_layer(x)
        output = self.relu(output)
        output = self.hidden_layer_1(output)
        output = self.relu(output)
        output = self.hidden_layer_2(output)
        output = self.relu(output)
        output = self.hidden_layer_3(output)
        output = self.relu(output)
        output = self.hidden_layer_4(output)
        output = self.relu(output)
        output = self.output_layer(output)
#         output = self.relu(output)
        
        return output
    
model = Regression_NN(input_dim=len(indep))

def get_NN_embeddings(df):
    scaler = MinMaxScaler()
    scaler.fit(df)

    df = scaler.transform(df)    
    
    model_path = f"../output_models/new__regression_nn_fold_0.bin"
    print(f"{0} : {model_path}")
    model.load_state_dict(torch.load(model_path))
    
    prediction = model(torch.tensor(df, dtype=torch.float))    
    pred = prediction.detach().numpy()#.reshape(-1)
    
    pred = scaler.inverse_transform(pred)
    
    NN_embeddings = pd.DataFrame(pred, columns = [f"nn_embed_{i}" for i in range(len(indep))])
    
    
    
    return NN_embeddings

train_nn_embed_df = get_NN_embeddings(df=train_df[indep])
train_nn_embed_df.shape

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df.to_pickle("../data/train_df_interim.pickle")
test_df.to_pickle("../data/test_df_interim.pickle")

train_df.to_csv("../data/train_df_interim.csv", index=False)
test_df.to_csv("../data/test_df_interim.csv", index=False)